# UCLQFF 2025 - Battleships 
###### Shamith Guniyangodage

## Introduction
Quantum Battleships is played on a 10x10 (can be n by n) grid.  
On this "sea" grid there are a number of unseen ships, The user can select a square to bomb, if a part of a ship is occupying this square it is destroyed, the user must bomb every occupied square on the sea to win.

The user also has access to a limited number of "stealth points". Using these points, the user can probe a specific square when bombing it.

However, bombing a grid exposes you to being seen by the ships, if seen you lose a life.

In [2]:
#install ipywidgets to play quantum battleships inline
#pip install ipywidgets 

In [3]:
#import necessary packages
#----------------------------

# packages for frontend user interface
import ipywidgets as widgets
from IPython.display import display, clear_output

#packages needed for game logic:
import random
import numpy as np

#access credentials from credentials.json:
import json

#Qiskit packages:
import matplotlib.pyplot as plt
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator

from qiskit.circuit import Parameter

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector

from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.visualization import plot_distribution

In [4]:
# Import IBM keys
with open('credentials.json') as f:
    creds = json.load(f)

api_key = creds['api_key']
crn = creds['crn']

print("Credential status: loaded")

QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=api_key,
    instance=crn,
    set_as_default=True,
    overwrite=True,
)

#sim backend
#backend = AerSimulator()

Credential status: loaded


## Quantum Zeno effect

Observing a quantum system/state frequently stops its wavefunction from evolving in time, effectively freezing it. We can use this to probe a "bomb".

If the bomb exists it collapses the wavefunction, therefore by continuosuly rotating around the X or Y axis in small nudges, (using Rx and Ry gates) we can have the wavefunction constantly collapse to 0, however if the bomb isnt there, after N rotations the bit flips and collapses to 1. 

## Implementation 
### 1. Game Class
- Implement the board as an n by n array 
- Given parameters of the number of ships wanted on the board and the possible sizes of the ships add to the board (1 = ship component exists in that square)
- helper member functions to display the current state of the board

- provide logic to peek and bomb squares on the grid


In [5]:
class Game:
    def __init__(self, board_size, ship_sizes, sampler, pass_manager):
        """
        Game class constructor, takes in a board size N and the number of ships on the board 
        and constructs a 2d np array with value 1 if a ship is present.
        """
        # Board attributes
        self.size = board_size
        self.__board = np.zeros((board_size, board_size), dtype=int)
        # Qiskit Tools
        self.sampler = sampler
        self.pm = pass_manager
        # Ships Attributes and win conditions
        self.ship_id_counter = 1
        self.total_ship_segments = sum(ship_sizes)
        self.found_ship_segments = 0
        self.player_board = np.full((board_size, board_size), -1)
        self.is_over = False

        #Place down hsips randomly on the board as part of the intialization of the Game class
        ships_placed = self._place_ships(ship_sizes)
        print(f"Ships placed on board: {ships_placed}")

    def _place_ships(self, ship_sizes):
        """randomly places ships on number and sizes of ships determined by caller"""
        ships = 0
        # NB, d_<direction> is just a diagonal placement of a ship
        orientations =(['horizontal', 'vertical',
                        'd_down_right', 'd_up_right',
                        'd_down_left', 'd_up_left'])
        # For the list of different ship sizes go through and attempt to add them to the board
        for length in ship_sizes:
            current_ship_id = self.ship_id_counter
            placed = False
            for i in range(self.size**2):   # === INITIALISING A LARGE BOARD WILL TAKE A WHILE
                # Choose random orientation and starting point of the ship
                orientation = random.choice(orientations)
                row = np.random.randint(0, self.size)
                col = np.random.randint(0, self.size)
                # Attempt to add a ship to the board
                if self.add_ship(length, row, col, orientation, current_ship_id):
                    placed = True
                    ships += 1
                    self.ship_id_counter += 1
                    break
            #If there is no position in which a ship of size L can be placed, dont place anything down
            # and carry on the game with 1 less ship than intended
            if not placed:
                print(f"Error: Could not place ship | length = {length}")
        return ships

    def add_ship(self, length, row, col, orientation, ship_id):
        """Tries to add a ship with given parameters to the board."""                    
        coords = [] 
        # Check if orientation and size fit 
        match orientation:
            case 'horizontal':
                if col + length > self.size: return False
                for i in range(length): coords.append((row, col + i))
            case 'vertical':
                if row + length > self.size: return False
                for i in range(length): coords.append((row + i, col))
            case 'd_down_right':
                if row + length > self.size or col + length > self.size: return False
                for i in range(length): coords.append((row + i, col + i))
            case 'd_up_right':
                if row - length < -1 or col + length > self.size: return False
                for i in range(length): coords.append((row - i, col + i))
            case 'd_down_left':
                if row + length > self.size or col - length < -1: return False
                for i in range(length): coords.append((row + i, col - i))
            case 'd_up_left':
                if row - length < -1 or col - length < -1: return False
                for i in range(length): coords.append((row - i, col - i))
            case _:
                print(f"Error: Unknown orientation '{orientation}'")
                return False
        
        # Ships should be placed on top of one another so check for this:

        #Check ALL coordinates first.
        for r, c in coords:
            if self.__board[r][c] >= 1:
                return False # Found an overlap, fail immediately.

        # If there are no overlap of ships place the ship down
        for r, c in coords:
            self.__board[r][c] = ship_id

        return True 

    def check_win_condition(self):
        """Checks if all ship segments have been found."""
        return self.found_ship_segments == self.total_ship_segments

    # -------------------------------------------------------------------------- <<< QUANTUM CIRCUIT 

    def _build_zeno_circuit(self, N, has_bomb):
        """Internal function to build the correct quantum circuit."""
        print("building and transpiling quantum circuit")
        theta = np.pi / N
        
        if has_bomb:
            # "With Bomb" circuit (ship exists at this location)
            qr = QuantumRegister(1, 'q'); cr = ClassicalRegister(N, 'c') 
            qc = QuantumCircuit(qr, cr)
            for i in range(N):
                qc.ry(theta, 0)
                qc.measure(qr[0], cr[i]) 
            return qc
        else:
            # "No Bomb" circuit (ship doesnt exist at this location)
            qr = QuantumRegister(1, 'q'); cr = ClassicalRegister(1, 'c')
            qc = QuantumCircuit(qr, cr)
            for i in range(N):
                qc.ry(theta, 0)
            qc.measure(qr[0], cr[0])
            return qc

    # Bomb a square
    def peek_square(self, player, row, col, N):
        """Probes a square and updates the player's stats."""
       
        if self.player_board[row][col] != -1:
            return "ALREADY_PEEKED", 0

        if player.stealth_pnts < N:
            return "BUDGET_LOW", 0
        
        player.stealth_pnts -= N
        is_ship_here = (self.__board[row][col] > 0)
        ship_id = self.__board[row][col]

        circuit = self._build_zeno_circuit(N=N, has_bomb=is_ship_here)
        print("Running transpiled circuit...")
        transpiled_circuit = self.pm.run(circuit)
        job = self.sampler.run([transpiled_circuit], shots=1)
        result = job.result()
        measurement_str = list(result[0].data.c.get_counts().keys())[0]
        
        if is_ship_here:
            safe_outcome_str = '0' * N
            if measurement_str == safe_outcome_str:
                player.safe_detections += 1
                self.found_ship_segments += 1
                self.player_board[row][col] = ship_id
                return "PING!", ship_id
            else:
                player.explosions += 1
                player.lives -= 1
                self.found_ship_segments += 1  
                self.player_board[row][col] = ship_id
                return "HIT!", ship_id
        else:
            # "MISS outcome"
            if measurement_str == '1':
                self.player_board[row][col] = 0 
                return "MISS", 0
            else:
                # A '0' result on an empty square is due to hardware noise
                self.player_board[row][col] = 0 
                return "PING! (Noise?)", 0
            
    def show_board(self):
        """prints out board state"""
        print(self.__board)


### 2. Player Class
- Instantiate with a number of lives
- Instantiate with a number of stealth points

- member functions to show stats and return e/v score

In [6]:
class Player:
    def __init__(self, lives, total_stealth_budget):
        self.lives = lives
        self.stealth_pnts = total_stealth_budget
        
        # Stats for the E.V. Score
        self.safe_detections = 0
        self.explosions = 0
        
    def show_stats_text(self):
        """Returns a formatted string of the player's stats for the UI."""
        ev_score = 0.0
        if self.explosions > 0:
            ev_score = self.safe_detections / self.explosions
        
        return (f"Lives: {self.lives} | "
                f"Stealth Budget: {self.stealth_pnts} | "
                f"Succesfull Bombings: {self.safe_detections} | "
                f"Detected and Hit: {self.explosions} | "
                f"E.V. Score: {ev_score:.2f}")

    def calculate_ev_score(self):
        if self.explosions == 0:
            return self.safe_detections  # Avoid divide-by-zero
        return self.safe_detections / self.explosions


## GAME SETTINGS - FEEL FREE TO TRY DIFFERENT CONFIGURATIONS!
- lives = 3
- ship sizes = 2-5 squares in length
- board size = 10 x 10 
- stealth budget = 2000 points

In [7]:
# use simulation instead of IBM cloud
backend = AerSimulator() 

# uncomment this to connect to the real IBM hardware:
# -- bear in mind you need to add a credentials.json file with your CRN and IBM cloud token :)
# ALSO BEAR IN MIND THIS WILL MAKE THE GAME WAY SLOWER
# print("Connecting to IBM Quantum...")
# service = QiskitRuntimeService()
# backend = service.least_busy(operational=True, simulator=False)
# print(f"Connected to backend: {backend.name}")

sampler = Sampler(mode=backend)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

lives = 5
ships_list = [5, 6, 4, 3, 3, 2] 
board_size = 10
stealth_pnts = 2000



## CREATE FRONTEND
- GUI interface to make this game more interactive
- Uses inline ipywidgets

In [ ]:
# create widgets for front end GUI
grid_buttons = []
button_grid = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(10, 40px)"))
stats_output = widgets.Output()     # For showing player stats
result_output = widgets.Output()    # For showing the result of a peek
n_slider = widgets.IntSlider(value=10, min=1, max=stealth_pnts/10, step=1, description='Stealth level:')
game_over_output = widgets.Output() # For showing the game over message

#define what pressing on a square does
def on_button_clicked(b):
    print("button pressed")
    # Get click info
    row, col = b.coords
    N = n_slider.value
    
    # Check Game State
    if player.lives <= 0:
        with game_over_output:
            clear_output()
            print("GAME OVER! You are out of lives. Restart the cell to play again.")
        return
    
    # Check if game is already won
    if game.is_over:
        return

    # peek the square to see if
    result_key , data = game.peek_square(player, row, col, N)
    
    # Update UI based on the result
    result_text = ""
    match result_key:
        case "PING!":
            ship_id = data
            result_text = f"PING! Ship {ship_id} detected - component succesfully destroyed"
            b.description = str(ship_id)
            b.button_style = 'success'; b.icon = ''
            b.disabled = True

        case "HIT!":
            ship_id = data
            result_text = f"HIT! Ship {ship_id} detected your presence and fired back - You lost a life!"
            b.button_style = 'danger'; b.icon = 'exclamation-triangle'
            b.disabled = True

        case "MISS":
            result_text = "MISS. No ships, just ocean"
            b.button_style = 'primary'; b.description = ''; b.icon = 'water'
            b.disabled = True

        case "BUDGET_LOW":
            result_text = f"Not enough stealth budget! You have {player.stealth_pnts}, but need {N}."
        
        case "ALREADY_PEEKED":
            ship_id = data
            if ship_id == 0:
                result_text = "You already bombed here. It's empty."
            else:
                result_text = f"You already bombed here. (It's Ship {ship_id})."
            
        case "PING! (Noise?)":
            result_text = "MISS (due to noise)."
            b.button_style = 'info'; b.description = ''; b.icon = 'check'
            b.disabled = True
    

    # Check for win condition
    if (result_key == "PING!" or result_key == "HIT!"):
        if game.check_win_condition():
            with game_over_output:
                clear_output()
                print(f"*** YOU WIN! ***")
                print(f"You found all {game.total_ship_segments} ship segments!")
                print(f"Final E.V. Score: {player.calculate_ev_score():.2f}")
            game.is_over = True
            return 

    # Update the text displays
    with result_output:
        clear_output()
        print(f"Bombed ({row},{col}) with Stealth={N}... {result_text}")
    
    with stats_output:
        clear_output()
        print(player.show_stats_text())
        
    # Check for game over
    if player.lives <= 0:
        with game_over_output:
            clear_output()
            print("GAME OVER! You ran out of lives. Restart the cell to play again.")
        game.is_over = True
            
    if player.stealth_pnts <= 0:
        with game_over_output:
            clear_output()
            print("GAME OVER! You ran out of peek budget. Restart the cell to play again.")
        game.is_over = True 


# initialize the game: 
print("Initializing Game...")

player = Player(lives=lives, total_stealth_budget=stealth_pnts)
game = Game(board_size=board_size, ship_sizes=ships_list, sampler=sampler, pass_manager=pm)

#BUILD AND DISPLAY THE UI 

# Create the 10x10 grid of buttons
for r in range(game.size):
    for c in range(game.size):
        btn = widgets.Button(description=f'?', layout=widgets.Layout(width='40px', height='40px'))
        btn.coords = (r, c)
        btn.on_click(on_button_clicked)
        grid_buttons.append(btn)
        
button_grid.children = grid_buttons

# Display all the widgets
print(f"{ships_list} | {np.sum(ships_list)}")
print("--- QUANTUM BATTLESHIPS ---")
display(stats_output, n_slider, button_grid, result_output, game_over_output)

# Initialize the stats display for the first time
with stats_output:
    clear_output()
    print(player.show_stats_text())
    
# You can un-comment this to cheat and see the board state :)
#game.show_board()



Initializing Game...
Ships placed on board: 6
[5, 6, 4, 3, 3, 2] | 23
--- QUANTUM BATTLESHIPS ---


Output()

IntSlider(value=10, description='Stealth level:', max=200, min=1)

GridBox(children=(Button(description='?', layout=Layout(height='40px', width='40px'), style=ButtonStyle()), Bu…

Output()

Output()

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 5 0 0 0]
 [0 0 0 3 0 0 2 5 4 0]
 [0 0 3 0 0 2 0 4 5 0]
 [0 3 0 0 2 0 4 0 1 0]
 [0 0 0 2 0 0 0 1 0 6]
 [0 0 2 0 0 0 1 0 0 6]
 [0 2 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
